We are looking at the problem of creating an optimal revision schedule for students studying towards their exams (A-Levels, AS, GCSEs etc.)

**Schedule Space:** The phrase we will use to describe the availability of time between now and the final exam date, the different subjects that can be studied in that time and the different activities that can be completed.

We can accurately determine how much effort is needed for a student to raise their score from their current score to their target score. All we need is their learning speed which is largely a stastistical approximation problem.

**Permissable Effort Units:** A measurement used to describe the amount of effort a student requires for each exam to reach their target grade.

We want to achieve the permissable effort units required in the minimal amount of time. If this allows us padded time, then we can do aditional studying to maximize our confidence in subjects, using a mathematical approximation on priorities.

However, optimizing the schedule space (when to do that effort and on what) is by no means a linear operation. When completing an action, achieved permissable effort units might change depending on: time to exam, loss of learning curve (how does retention of the subject fall after it's learned and how in each subsequent learning), number of hours spent revising already that day.

In other words, actions aren't homogenous, and we can't apply linear optimisation techniques to work out the optimal revision schedule.


If you think about the problem, an exhaustive search would be computationally infeasible

Example:
If you have 400 hours (200 study segments) for Maths and 200 hours (100 study segments) for English.
The revision schedule consists of 60 possible days each with 6 segments = 360 study segments
This is an extremely large problem to solve in relation to pure permutations. Therefore, we will look at a stochastical optimisation approximation technique.

# Simulated Annealing

We looked at multiple other optimisation techniques such as Particle Swarm Optimisation (PSO) and Genetic Algorithm. Genetic Algorithm is slightly too simple for our use case and PSO works better when the starting point is randomised across a feature space with little knowledge of where a useful optima might lie.

In our use case, we have a good starting point for our revision schedule so we decided to use simulated annealing.

### State Representation

The schedule space is the days from tomorrow (we don't include today for simplicity, but this is something that can be altered later) to the final exam date (presumably the student wants to stop studying after their final exam).

Each day is split into 24 (8am-8pm) 30 minute components but we don't order these i.e. {'Maths' : 5, 'Chemistry' : 5} is the same as {'Chemistry' : 5, 'Maths' : 5}. However, each block of activity must last for a minimum of 2 hours to prevent excessive changing of subjects.

Each activity will be assigned to a date, with a corresponding achieved PEU.

States can be modified by:
Increasing/decreasing lengths of activity
and adding the decreases to other days
Maintaining a minimal revision time of 2 hours

### Objective Function

Our goal is to minimize the amount of hours it takes to achieve the required PEUs. We can calculate the required PEUs using the following code:

In [4]:
# Calculate the amount of required 'permissable effort units' to exert on each exam
def time_spend(subjects):
    # The exertion required for each learning speed to raise a point
    learning_speed_values = {
    'very_fast': 1,
    'fast': 2,
    'medium': 3,
    'slow': 4,
    }

    time_to_spend = {}
    for subject in subjects:
        learning_speed = learning_speed_values.get(subject['learning_speed'], 0)
        time_to_spend[subject['subject_name']] = (subject['target_score'] - subject['current_score'])*learning_speed
    
    return time_to_spend

We can then calculate the achieved permissable units

In [20]:
# Calculate the achieved 'permissable effort units'
def calculate_achieved_peus(schedule, subjects):
    required_permissable_effort_units = time_spend(subjects)
    achieved_permissable_effort_units = {subject['subject_name']: 0 for subject in subjects}
    
    for dates,days in schedule.items():
        full_study_that_day = 0
        full_study_that_day = sum(hours for hours in days.values() if hours != 'Exam Day')
            
        for subject, hours in days.items():
            if hours == 'Exam Day':
                continue
          
            study_amount_deterioration=value_per_day[full_study_that_day]
            for subject_take in subjects:
                if subject_take['subject_name'] == subject:
                    exam_date = subject_take['exam_date']
            time_to_exam = (exam_date - dates).days
            if time_to_exam > 30:
                time_amount_deterioration = value_vs_time_to_exam['more_than_30_days']
            elif time_to_exam > 15:
                time_amount_deterioration = value_vs_time_to_exam['between_30_days_and_15_days']
            elif time_to_exam > 7:
                time_amount_deterioration = value_vs_time_to_exam['between_15_days_and_7_days']
            else:
                time_amount_deterioration = value_vs_time_to_exam['under_7_days']
            achieved_permissable_effort_units[subject] += (hours * study_amount_deterioration * time_amount_deterioration)
    
    return achieved_permissable_effort_units
            
    

We will also need to define the actual objective function

In [88]:
def objective_function(schedule,subjects):
    achieved_permissable_effort_units = calculate_achieved_peus(schedule,subjects)
    required_permissable_effort_units = time_spend(subjects)
    for subject, hours in required_permissable_effort_units.items():
        if hours > achieved_permissable_effort_units[subject]:
            return math.inf #if the required PEUs aren't achieved
    return sum(sum(hours for hours in days.values() if hours != 'Exam Day') for days in schedule.values()) #This is what we want to minimize

### Initial Solution

In [22]:
from datetime import datetime, timedelta

import math
import pandas as pd


maximum_revision_time = 5 # Our assumption on the maximum number of hours revision

def initial_solution(subjects):
    #Initialize initial solution
    initial_solution = {}
    # Extract the exam dates
    exam_dates = {}
    for subject in subjects:
        exam_dates[subject['subject_name']] = subject['exam_date']
    
    # Main loop to add hours in while achieved PEUs (through the schedule) are less than the required PEUs
    required_peus = time_spend(subjects)
    achieved_peus = calculate_achieved_peus(initial_solution,subjects)
    for subject in subjects:
        date = exam_dates[subject['subject_name']] - timedelta(days=1) #Initialize date on first day before the exam date
        current_date = datetime.now()
        #1st pass attempt at adding only 5 hours
        while required_peus[subject['subject_name']] > achieved_peus[subject['subject_name']] and date > current_date:
            if date not in exam_dates.values(): #No study on exam days
                if date not in initial_solution:
                    initial_solution[date] = {}
                
                if required_peus[subject['subject_name']] - achieved_peus[subject['subject_name']] >= maximum_revision_time:    
                    initial_solution[date][subject['subject_name']] = maximum_revision_time
                elif required_peus[subject['subject_name']] - achieved_peus[subject['subject_name']] < maximum_revision_time: 
                    initial_solution[date][subject['subject_name']] = math.ceil(required_peus[subject['subject_name']] - achieved_peus[subject['subject_name']])
            achieved_peus = calculate_achieved_peus(initial_solution,subjects)
            date -= timedelta(days=1)
    
    #2nd pass attempt adding remaining amounts
    for subject in subjects:
        if achieved_peus[subject['subject_name']] < required_peus[subject['subject_name']]:
            date = exam_dates[subject['subject_name']] - timedelta(days=1)
            while required_peus[subject['subject_name']] > achieved_peus[subject['subject_name']] and date > current_date:
                if date not in exam_dates.values():
                    if date not in initial_solution:
                        initial_solution[date] = {}
                    if sum(initial_solution[date].values()) < 12:
                        if sum(initial_solution[date].values()) < 11:  # Check if the sum of hours is less than 12
                            remaining_hours = 11 - sum(initial_solution[date].values())
                            required_hours = math.ceil(required_peus[subject['subject_name']] - achieved_peus[subject['subject_name']])
                            if subject['subject_name'] not in initial_solution[date]:
                                initial_solution[date][subject['subject_name']] = 0
                            initial_solution[date][subject['subject_name']] += min(remaining_hours, required_hours)
                achieved_peus = calculate_achieved_peus(initial_solution,subjects)
                date -= timedelta(days=1)
                
               
    for subject, exam_date in exam_dates.items():
        if exam_date not in initial_solution:
            initial_solution[exam_date] = {}
        initial_solution[exam_date][subject] = 'Exam Day'
        
    start_date = min(initial_solution.keys())
    end_date = max(initial_solution.keys())
    
    date_range = pd.date_range(start=start_date, end=end_date)
    
    datetime_range = date_range.to_pydatetime()

    for date in datetime_range:
        if date not in initial_solution:
            initial_solution[date] = {}
            for subject in subjects:
                if subject['exam_date'] > date:
                    
                    initial_solution[date][subject['subject_name']] = 2
                    
    #return (calculate_achieved_peus(initial_solution,subjects),required_peus)
    
    return {k: initial_solution[k] for k in sorted(initial_solution)}

### Neighbourhood Function

In [23]:
def get_exam_date(subjects, subject_name):
    for subject in subjects:
        if subject['subject_name'] == subject_name:
            return subject['exam_date']
    # Return None if the subject_name is not found in the list
    return None

In [24]:
# Function to check if a given day is an exam day
def is_exam_day(schedule, day):
    for subjects in schedule.get(day, {}).values():
        if subjects == 'Exam Day':
            return True
    return False

In [102]:
def move_function(schedule):
    # Get the list of dates in the schedule
    dates = [day for day in schedule.keys() if not is_exam_day(schedule, day)]
    
    # Shuffle the dates randomly
    random.shuffle(dates)
    
    returned_schedule=0
    returned_resp=0
        
    for date in dates:
        # Check if the date is not an exam day and has study hours
        if date not in schedule or sum(schedule[date].values()) == 0:
            continue
        
        # Get the subjects studied on the current date
        subjects_studied = list(schedule[date].keys())
        
        # Choose a random subject from the current date
        selected_subject = random.choice(subjects_studied)
        
        # Get the study hours for the selected subject on the current date
        study_hours = schedule[date][selected_subject]
        
        if study_hours == 0:
            continue
        
        # Choose a random number of hours to move to another date
        hours_to_move = random.randint(1, study_hours)
        
        # Get the exam date
        exam_date = get_exam_date(subjects,selected_subject)
        
        # Get a list of possible destination dates before the exam day
        destination_dates = [d for d in dates if d < exam_date]
        
        # Shuffle the destination dates randomly
        random.shuffle(destination_dates)
        
        # Find the first destination date where the total study hours won't exceed 12
        destination_date = None
        for dest_date in destination_dates:
            if dest_date in schedule:
                total_hours_on_dest_date = sum(schedule[dest_date].values())
            else:
                total_hours_on_dest_date = 0
            
            if total_hours_on_dest_date + hours_to_move <= 12:
                destination_date = dest_date
                break
        
        # If a valid destination date is found, move the hours
        if destination_date is not None:
            # Remove the hours from the current date
            schedule[date][selected_subject] -= hours_to_move
            
            # Add the hours to the destination date
            if destination_date not in schedule:
                schedule[destination_date] = {}
            if selected_subject not in schedule[destination_date]:
                schedule[destination_date][selected_subject] = 0
            schedule[destination_date][selected_subject] += hours_to_move
            
            returned_resp=f"Moved {hours_to_move} hours from {selected_subject} on {date} to {destination_date}."
            returned_schedule=schedule
            # Exit the loop as we have successfully moved the hours
            break
        else:
            continue
    
    if returned_resp==0:
        returned_resp= f"Could not find a valid destination date to move hours"
    return returned_schedule

        

In [62]:
def add_function(schedule):
    
    # Get the list of dates and we can look for any date from now to the day before final exam
    available_days = [datetime(datetime.now().date().year, (datetime.now() + timedelta(days=i)).date().month, (datetime.now() + timedelta(days=i)).date().day) for i in range((max(schedule.keys()) - datetime.now()).days+1) if not is_exam_day(schedule, datetime(datetime.now().date().year, (datetime.now() + timedelta(days=i)).date().month, (datetime.now() + timedelta(days=i)).date().day))]
    
    # Shuffle the dates randomly
    random.shuffle(available_days)
    
    for date in available_days:
        # Get the current hours on the random date
        if date not in schedule:
            current_hours=0
        else:
            current_hours = sum(schedule[date].values())
        
        # Continue if date is full
        if current_hours >= 12:
            continue
            
        # Get the subjects studied on the current date
        subjects_studied = [subject['subject_name'] for subject in subjects]
        
        random.shuffle(subjects_studied)
            
        # Generate a random number of study hours (between 1 and 12)
        if current_hours == 0:
            random_hours = random.randint(1, 12)
        else:
            random_hours = random.randint(1, current_hours)
        
        for subject in subjects_studied:
            # Get the exam date for the selected subject
            exam_date = get_exam_date(subjects,subject)
            if date > exam_date:
                continue
            else:
                if current_hours + random_hours > 12:
                    continue
                if date not in schedule:
                    schedule[date] = {}
                if subject not in schedule[date]:
                    schedule[date][subject] = random_hours
                    resp=f"Added {random_hours} to {subject} on {date}"
                    return schedule
                else:
                    schedule[date][subject] += random_hours
                    resp=f"Added {random_hours} to {subject} on {date}"
                    return schedule

In [51]:
def minus_function(schedule):
    
    # Get the list of dates in the schedule
    dates = [day for day in schedule.keys() if not is_exam_day(schedule, day)]
    
    # Shuffle the dates randomly
    random.shuffle(dates)
    
    for d in dates:
    
        # Choose a random day from the available days
        date = random.choice(dates)
               
        # Get the subjects studied on the current date
        subjects_studied = list(schedule[date].keys())
        
        # Choose a random subject from the current date
        selected_subject = random.choice(subjects_studied)
        
        # Get the study hours for the selected subject on the current date
        study_hours = schedule[date][selected_subject]
        
        if study_hours == 0:
            continue
        else:
            # Choose a random number of hours to minus
            hours_to_minus = random.randint(1, study_hours)

            #Minus the hours
            schedule[date][selected_subject] -= hours_to_minus

            resp=f"Minused {hours_to_minus} from {date} for {selected_subject}"
            break

    return schedule

In [52]:
def swap_function(schedule):
    
    subject_list = list(subject for subjects_data in schedule.values() for subject in subjects_data)
    random.shuffle(subject_list)
    
    returned_resp = 0
    returned_schedule = 0
    
    subject_list1 = list(set(subject_list))
    
    for subject1 in subject_list1:
        exam_date1 = get_exam_date(subjects, subject1)
        available_days1 = [day for day in schedule.keys() if day < exam_date1 and not is_exam_day(schedule, day)]
        
        if not available_days1:
            continue
            
        subject_list2 = [subject for subject in subject_list if subject != subject1]


        for subject2 in subject_list2:
            exam_date2 = get_exam_date(subjects, subject2)
            available_days2 = [day for day in schedule.keys() if day < exam_date2 and not is_exam_day(schedule, day)]
    
            if not available_days2:
                continue
            
            available_days = [date for date in available_days1 if date in available_days2]
            
            random.shuffle(available_days)

            for d1 in available_days:
                if subject1 in schedule[d1] and schedule[d1][subject1] == 0 or subject1 not in schedule[d1]:
                    continue
                
                available_days3 = [date for date in available_days if date != d1]
                
                for d2 in available_days3:
                    if subject2 in schedule[d2] and schedule[d2][subject2] == 0 or subject2 not in schedule[d2]:
                        continue
                        
    
                    random_hours_to_swap = random.randint(1, schedule[d1][subject1])        
    
                    # Check if the second subject has enough study hours to swap
                    if random_hours_to_swap > schedule[d2][subject2]:
                        # Reduce the amount to be swapped to avoid negative values
                        random_hours_to_swap = schedule[d2][subject2]

                    # Reduce study hours
                    if subject1 in schedule[d1]:
                        schedule[d1][subject1] -= random_hours_to_swap
                        if schedule[d1][subject1] == 0:
                            del schedule[d1][subject1]
                            
                    if subject2 in schedule[d2]:
                        schedule[d2][subject2] -= random_hours_to_swap
                        if schedule[d2][subject2] == 0:
                            del schedule[d2][subject2]

                    if subject1 not in schedule[d2]:
                        schedule[d2][subject1] = random_hours_to_swap
                    else:
                        schedule[d2][subject1] += random_hours_to_swap
                        
                    if subject2 not in schedule[d1]:
                        schedule[d1][subject2] = random_hours_to_swap
                    else:
                        schedule[d1][subject2] += random_hours_to_swap

                    returned_resp = f"Swapped {random_hours_to_swap} from {subject1} on {d1} to {subject2} on {d2}"
                    break
                break
            break
        break
    
    return schedule

In [109]:
import random

def neighbourhood_function(schedule):
    random_number = random.random()
    if random_number < 0.25:
        return move_function(schedule) # Moving study hours from one day to another
    if random_number >= 0.25 and random_number < 0.65:
        return add_function(schedule) # Adding hours for a subject
    if random_number >= 0.65 and random_number < 0.75:
        return minus_function(schedule) # Reducing hours for a subject
    else:
        return swap_function(schedule) # Swapping subjects between days

In [110]:
def simulated_annealing(subjects, n_iterations, step_size, temp):
    # generate an initial starting point
    initial = initial_solution(subjects)
    # evaluate the initial starting point
    initial_eval = objective_function(initial, subjects)
    # initialize currents and bes
    curr, curr_eval = initial, initial_eval
    best, best_eval = initial, initial_eval
    
    # run the algorithm
    for i in range(n_iterations):
        candidate = curr
        # take a step
        for j in range(step_size):
            candidate = neighbourhood_function(candidate)
        # evaluate candidate
        candidate_eval = objective_function(candidate, subjects)
        # check for new best solution
        if candidate_eval < best_eval:
            # store new best point
            best, best_eval = candidate, candidate_eval
            # report progress
            print(best_eval)
        # difference between candidate and current point evaluation
        diff = candidate_eval - curr_eval
        # calculate temperature for current epoch
        t = temp / float(i + 1)
        # calculate metropolis acceptance criterion
        metropolis = math.exp(-diff / t)
        # check if we should keep the new point
        if diff < 0 or random.random() < metropolis:
            # store the new current point
            curr, curr_eval = candidate, candidate_eval
    return [best, best_eval]

In [66]:
value_per_day = {
    0 : 1,
    1 : 1,
    2 : 1,
    3 : 1,
    4 : 1,
    5 : 1,
    6 : 0.95,
    7 : 0.9,
    8 : 0.85,
    9 : 0.8,
    10 : 0.75,
    11 : 0.7,
    12 : 0.65,
    13 : 0.6 # At this point it starts to become deteriorating to study more
}

In [32]:
value_vs_time_to_exam = {
   'more_than_30_days' : 0.5,
   'between_30_days_and_15_days' : 0.7,
   'between_15_days_and_7_days' : 0.8,
   'under_7_days' : 1 
} # Assumption on how the value of the study changes depending on the time to the exam

In [33]:
subjects = [
    {'subject_name' : 'Maths',
     'current_score' : 67,
     'target_score' : 90,
     'learning_speed' : 'fast',
     'exam_date' : datetime(2023,10,28)
    },
    {'subject_name' : 'Chemistry',
     'current_score' : 54,
     'target_score' : 80,
     'learning_speed' : 'slow',
     'exam_date' : datetime(2023,9,12)
    },
    {'subject_name' : 'Physics',
     'current_score' : 72,
     'target_score' : 80,
     'learning_speed' : 'medium',
     'exam_date' : datetime(2023,9,10)
    },
]

In [112]:
simulated_annealing(subjects, 500, 2, 6)

300


[{datetime.datetime(2023, 8, 15, 0, 0): {'Chemistry': 8, 'Physics': 4},
  datetime.datetime(2023, 8, 16, 0, 0): {'Physics': 9, 'Chemistry': 3},
  datetime.datetime(2023, 8, 17, 0, 0): {'Chemistry': 5,
   'Physics': 6,
   'Maths': 1},
  datetime.datetime(2023, 8, 18, 0, 0): {'Physics': 11, 'Maths': 1},
  datetime.datetime(2023, 8, 19, 0, 0): {'Maths': 1, 'Chemistry': 11},
  datetime.datetime(2023, 8, 20, 0, 0): {'Chemistry': 5, 'Maths': 7},
  datetime.datetime(2023, 8, 21, 0, 0): {'Chemistry': 10, 'Physics': 2},
  datetime.datetime(2023, 8, 22, 0, 0): {'Maths': 7,
   'Chemistry': 4,
   'Physics': 1},
  datetime.datetime(2023, 8, 23, 0, 0): {'Chemistry': 12},
  datetime.datetime(2023, 8, 24, 0, 0): {'Physics': 6, 'Chemistry': 6},
  datetime.datetime(2023, 8, 25, 0, 0): {'Maths': 12},
  datetime.datetime(2023, 8, 26, 0, 0): {'Chemistry': 9, 'Maths': 3},
  datetime.datetime(2023, 8, 27, 0, 0): {'Physics': 3, 'Maths': 9},
  datetime.datetime(2023, 8, 28, 0, 0): {'Chemistry': 7,
   'Physics'

In [82]:
calculate_achieved_peus(schedule,subjects)

{'Maths': 150.28000000000003,
 'Chemistry': 108.39000000000001,
 'Physics': 24.549999999999997}

In [83]:
time_spend(subjects)

{'Maths': 46, 'Chemistry': 104, 'Physics': 24}

In [81]:
schedule = {datetime(2023, 8, 15, 0, 0): {'Chemistry': 4, 'Maths': 5},
  datetime(2023, 8, 16, 0, 0): {'Chemistry': 11, 'Physics': 1},
  datetime(2023, 8, 17, 0, 0): {'Chemistry': 1, 'Physics': 4},
  datetime(2023, 8, 18, 0, 0): {'Chemistry': 5},
  datetime(2023, 8, 19, 0, 0): {'Chemistry': 8, 'Maths': 4},
  datetime(2023, 8, 20, 0, 0): {'Chemistry': 9},
  datetime(2023, 8, 21, 0, 0): {'Chemistry': 1, 'Maths': 4},
  datetime(2023, 8, 22, 0, 0): {'Chemistry': 9},
  datetime(2023, 8, 23, 0, 0): {'Physics': 5,
   'Maths': 1,
   'Chemistry': 4},
  datetime(2023, 8, 24, 0, 0): {'Chemistry': 8},
  datetime(2023, 8, 25, 0, 0): {'Chemistry': 2, 'Physics': 3},
  datetime(2023, 8, 26, 0, 0): {'Chemistry': 7},
  datetime(2023, 8, 27, 0, 0): {'Chemistry': 3, 'Maths': 2},
  datetime(2023, 8, 28, 0, 0): {'Chemistry': 2, 'Physics': 2},
  datetime(2023, 8, 29, 0, 0): {'Chemistry': 4, 'Maths': 1},
  datetime(2023, 8, 30, 0, 0): {'Chemistry': 11},
  datetime(2023, 8, 31, 0, 0): {'Chemistry': 10},
  datetime(2023, 9, 1, 0, 0): {'Chemistry': 6, 'Physics': 1},
  datetime(2023, 9, 2, 0, 0): {'Chemistry': 10, 'Physics': 0},
  datetime(2023, 9, 3, 0, 0): {'Chemistry': 8, 'Physics': 3},
  datetime(2023, 9, 4, 0, 0): {'Chemistry': 1,
   'Physics': 4,
   'Maths': 2},
  datetime(2023, 9, 5, 0, 0): {'Chemistry': 8, 'Physics': 0},
  datetime(2023, 9, 6, 0, 0): {'Chemistry': 5,
   'Physics': 5,
   'Maths': 1},
  datetime(2023, 9, 7, 0, 0): {'Chemistry': 6, 'Physics': 1},
  datetime(2023, 9, 8, 0, 0): {'Chemistry': 6, 'Physics': 5},
  datetime(2023, 9, 9, 0, 0): {'Chemistry': 11, 'Physics': 1},
  datetime(2023, 9, 10, 0, 0): {'Physics': 'Exam Day'},
  datetime(2023, 9, 11, 0, 0): {'Chemistry': 11},
  datetime(2023, 9, 12, 0, 0): {'Chemistry': 'Exam Day'},
  datetime(2023, 9, 13, 0, 0): {'Maths': 3},
  datetime(2023, 9, 14, 0, 0): {'Maths': 2},
  datetime(2023, 9, 15, 0, 0): {'Maths': 2},
  datetime(2023, 9, 16, 0, 0): {'Maths': 4},
  datetime(2023, 9, 17, 0, 0): {'Maths': 5},
  datetime(2023, 9, 18, 0, 0): {'Maths': 9},
  datetime(2023, 9, 19, 0, 0): {'Maths': 4},
  datetime(2023, 9, 20, 0, 0): {'Maths': 7},
  datetime(2023, 9, 21, 0, 0): {'Maths': 4},
  datetime(2023, 9, 22, 0, 0): {'Maths': 0},
  datetime(2023, 9, 23, 0, 0): {'Maths': 6},
  datetime(2023, 9, 24, 0, 0): {'Maths': 12},
  datetime(2023, 9, 25, 0, 0): {'Maths': 3},
  datetime(2023, 9, 26, 0, 0): {'Maths': 10},
  datetime(2023, 9, 27, 0, 0): {'Maths': 8},
  datetime(2023, 9, 28, 0, 0): {'Maths': 1},
  datetime(2023, 9, 29, 0, 0): {'Maths': 8},
  datetime(2023, 9, 30, 0, 0): {'Maths': 4},
  datetime(2023, 10, 1, 0, 0): {'Maths': 4},
  datetime(2023, 10, 2, 0, 0): {'Maths': 7},
  datetime(2023, 10, 3, 0, 0): {'Maths': 5},
  datetime(2023, 10, 4, 0, 0): {'Maths': 2},
  datetime(2023, 10, 5, 0, 0): {'Maths': 2},
  datetime(2023, 10, 6, 0, 0): {'Maths': 7},
  datetime(2023, 10, 7, 0, 0): {'Maths': 5},
  datetime(2023, 10, 8, 0, 0): {'Maths': 2},
  datetime(2023, 10, 9, 0, 0): {'Maths': 2},
  datetime(2023, 10, 10, 0, 0): {'Maths': 7},
  datetime(2023, 10, 11, 0, 0): {'Maths': 9},
  datetime(2023, 10, 12, 0, 0): {'Maths': 2},
  datetime(2023, 10, 13, 0, 0): {'Maths': 0},
  datetime(2023, 10, 14, 0, 0): {'Maths': 4},
  datetime(2023, 10, 15, 0, 0): {'Maths': 4},
  datetime(2023, 10, 16, 0, 0): {'Maths': 3},
  datetime(2023, 10, 17, 0, 0): {'Maths': 1},
  datetime(2023, 10, 18, 0, 0): {'Maths': 6},
  datetime(2023, 10, 19, 0, 0): {'Maths': 12},
  datetime(2023, 10, 20, 0, 0): {'Maths': 0},
  datetime(2023, 10, 21, 0, 0): {'Maths': 9},
  datetime(2023, 10, 22, 0, 0): {'Maths': 8},
  datetime(2023, 10, 23, 0, 0): {'Maths': 1},
  datetime(2023, 10, 24, 0, 0): {'Maths': 6},
  datetime(2023, 10, 25, 0, 0): {'Maths': 8},
  datetime(2023, 10, 26, 0, 0): {'Maths': 8},
  datetime(2023, 10, 27, 0, 0): {'Maths': 7},
  datetime(2023, 10, 28, 0, 0): {'Maths': 'Exam Day'},
  datetime(2023, 8, 14, 0, 0): {'Maths': 9, 'Chemistry': 1}}

In [84]:
sum(sum(hours for hours in days.values() if hours != 'Exam Day') for days in schedule.values()) #This is what we want to minimize

459

In [85]:
objective_function(schedule,subjects)

459